# criando csv principal

In [1]:
import pandas as pd
import numpy as np
from helpers import BRASIL_UFS ,get_uf_df

In [ ]:
dfs = [get_uf_df(uf) for uf in BRASIL_UFS]

In [6]:
big_df = pd.concat(dfs)

In [ ]:
print(len(big_df))
big_df.head()

In [9]:
def get_pp_lula(x,df):
    try:
        votos_por_municipio = df.groupby(by=["NM_MUNICIPIO", "NR_PARTIDO"]).agg(
            quantidade=("QT_VOTOS", "sum")
        )
        votos_lula = votos_por_municipio["quantidade"][x["NM_MUNICIPIO"]][13]
        votos_bolsonaro = votos_por_municipio["quantidade"][x["NM_MUNICIPIO"]][22]
        total_votos_validos = votos_lula + votos_bolsonaro
        return votos_lula / total_votos_validos , total_votos_validos
    except:
        print(x["NM_MUNICIPIO"], "não encontrado no dataframe de votos por municipio.")
        return 0.5


def get_pp_municipio(df):
    ndf = df.drop_duplicates(subset=["NM_MUNICIPIO"])
    ndf["PP_LULA"] = ndf.apply(lambda x: get_pp_lula(x, df)[0], axis=1)
    ndf["QT_VOTOS"] = ndf.apply(lambda x: get_pp_lula(x, df)[1], axis=1)
    return ndf.drop('NR_PARTIDO', axis=1)

pp_df = get_pp_municipio(big_df)
pp_df

/tmp/ipykernel_9694/22180300.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ndf["PP_LULA"] = ndf.apply(lambda x: get_pp_lula(x, df)[0], axis=1)
/tmp/ipykernel_9694/22180300.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ndf["QT_VOTOS"] = ndf.apply(lambda x: get_pp_lula(x, df)[1], axis=1)


,SG_UF,NM_MUNICIPIO,CD_MUNICIPIO,QT_VOTOS,PP_LULA
0,RO,GUAJARÁ-MIRIM,19,19839,0.344675
301,RO,NOVA MAMORÉ,434,13123,0.220757
1250,RO,PORTO VELHO,35,261935,0.353660
3607,RO,ITAPUÃ DO OESTE,493,4378,0.375057
4163,RO,JI-PARANÁ,51,68126,0.245295
...,...,...,...,...,...
51990,GO,SÃO JOÃO D'ALIANÇA,95974,5305,0.509896
52093,GO,COLINAS DO SUL,92126,2314,0.671132
52415,GO,OURO VERDE DE GOIÁS,95052,2912,0.406593
53009,GO,CAMPO LIMPO DE GOIÁS,93580,4701,0.359073


In [ ]:
pp_df.to_csv('pp_lula_by_municipio.csv', index=False)

## Salvamento do csv

In [ ]:
pp_df = pd.read_csv('pp_lula_by_municipio.csv')

# Dados por região

In [19]:
pp_df = pd.read_csv('pp_lula_by_municipio.csv')
regioes = pd.read_csv('regioes.csv')
len(pp_df), len(regioes)

(5297, 27)

In [20]:
com_regiao = pd.merge(pp_df, regioes, on='SG_UF', how='left')
com_regiao

5297


,SG_UF,NM_MUNICIPIO,CD_MUNICIPIO,QT_VOTOS,PP_LULA,REGIAO
0,RO,GUAJARÁ-MIRIM,19,19839,0.344675,Norte
1,RO,NOVA MAMORÉ,434,13123,0.220757,Norte
2,RO,PORTO VELHO,35,261935,0.353660,Norte
3,RO,ITAPUÃ DO OESTE,493,4378,0.375057,Norte
4,RO,JI-PARANÁ,51,68126,0.245295,Norte
...,...,...,...,...,...,...
5292,GO,SÃO JOÃO D'ALIANÇA,95974,5305,0.509896,Centro-Oeste
5293,GO,COLINAS DO SUL,92126,2314,0.671132,Centro-Oeste
5294,GO,OURO VERDE DE GOIÁS,95052,2912,0.406593,Centro-Oeste
5295,GO,CAMPO LIMPO DE GOIÁS,93580,4701,0.359073,Centro-Oeste


# Adicionando informações ao dataframe

## PIB

In [21]:
pib_df = pd.read_html('https://pt.wikipedia.org/wiki/Lista_de_munic%C3%ADpios_do_Brasil_por_PIB')[0]
pib_df = pib_df.rename(columns={"Produto Interno Bruto (em R$ 1 000)":"pib"})
pib_df['Município'] = pib_df['Município'].apply(str.upper)
pib_df = pib_df[['Município','pib']]
pib_df.head()

,Município,pib
0,SÃO PAULO,828 980 607
1,RIO DE JANEIRO,359 634 752
2,BRASÍLIA,286 943 782
3,BELO HORIZONTE,105 829 675
4,MANAUS,103 281 436


In [22]:
from helpers import get_nearest
pp_df = pd.read_csv('pp_lula_by_municipio.csv')

def get_item_from_df(municipe,df,item):
    try:
        pib = df[df['Município'] == municipe][item].values[0]
    except Exception as e:
        try:
            nearest = get_nearest(municipe,df['Município'])
            pib = df[df['Município'] == nearest][item].values[0]
        except Exception as e:
            print(municipe,item)
            return np.nan
    return pib

pp_df['pib'] = pp_df['NM_MUNICIPIO'].apply(lambda x: get_item_from_df(x,pib_df,'pib'))
pp_df.head()

,SG_UF,NM_MUNICIPIO,CD_MUNICIPIO,QT_VOTOS,PP_LULA,pib
0,RO,GUAJARÁ-MIRIM,19,19839,0.344675,984 586
1,RO,NOVA MAMORÉ,434,13123,0.220757,601 910
2,RO,PORTO VELHO,35,261935,0.353660,19 448 762
3,RO,ITAPUÃ DO OESTE,493,4378,0.375057,192 783
4,RO,JI-PARANÁ,51,68126,0.245295,4 057 626


### Salvamento com o PIB

In [23]:
pp_df.to_csv('pp_lula_by_municipio.csv', index=False)